In [ ]:
import mitiq
import numpy
from numpy import pi
from cirq import CircuitOperation, CXPowGate, CZPowGate, DensityMatrixSimulator, Simulator, Rx
from cirq.devices.noise_model import GateSubstitutionNoiseModel
from cirq import LineQubit, Circuit, CZ, CNOT, H, Ry, Rz
from cirq import Circuit, LineQubit, H, CNOT, CZ
from functools import partial
import numpy as np
from mitiq import Executor
from mitiq.pt import generate_pauli_twirl_variants
from mitiq.shadows.shadows_utils import fidelity
import cirq

mitiq.SUPPORTED_PROGRAM_TYPES.keys()

In [ ]:
def circular_ansatz_mirrored(N, reps=1, fix_2q=False):
    qubits = LineQubit.range(N)
    circuit = Circuit()
    for _ in range(reps):
        for q in qubits:
            circuit.append(H(q))
        for i in range(N):
            control = qubits[(i-1) % N]
            target = qubits[i]
            circuit.append(CNOT(control, target))
        for q in qubits:
            circuit.append(H(q))
        for i in range(N-1, -1, -1):
            control = qubits[(i-1) % N]
            target = qubits[i]
            circuit.append(CNOT(control, target))
    for q in qubits:
        circuit.append(H(q))
    return circuit


def get_noise_model(noise_level: float) -> GateSubstitutionNoiseModel:
    """Substitute each CZ and CNOT gate in the circuit
    with the gate itself followed by an Rx rotation on the output qubits.
    """
    rads = pi / 2 * noise_level
    def noisy_c_gate(op):
        if isinstance(op.gate, (CZPowGate, CXPowGate)):
            return CircuitOperation(
                Circuit(
                    op.gate.on(*op.qubits), 
                    Rx(rads=rads).on_each(op.qubits),
                ).freeze())
        return op

    return GateSubstitutionNoiseModel(noisy_c_gate)

def execute(circuit: Circuit, noise_level: float):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit."""
    return (
        DensityMatrixSimulator(noise=get_noise_model(noise_level=noise_level))
        .simulate(circuit)
        .final_density_matrix
    )

# Set the intensity of the noise
NOISE_LEVEL = 1e-1

In [ ]:
num_qubits = 12   
circuit = circular_ansatz_mirrored(num_qubits,2)

In [ ]:
# Compute the expectation value of the |0><0| observable
# in both the noiseless and the noisy setup
ideal_value = execute(circuit, noise_level=0.0)
noisy_value = execute(circuit, noise_level=NOISE_LEVEL)

In [ ]:
# Generate twirled circuits
NUM_TWIRLED_VARIANTS = 100
twirled_circuits = generate_pauli_twirl_variants(circuit, num_circuits=NUM_TWIRLED_VARIANTS)
# Average results executed over twirled circuits
# pt_vals = Executor(partial(execute, noise_level=NOISE_LEVEL)).evaluate(twirled_circuits)
pt_vals = Executor(partial(execute, noise_level=NOISE_LEVEL)).run(twirled_circuits)
mitigated_result = np.average(pt_vals, axis=0)

In [ ]:
a = cirq.density_matrix(ideal_value,atol=1e-6)

b = cirq.density_matrix(noisy_value,atol=1e-6)
c = cirq.density_matrix(mitigated_result,atol=1e-6)

fidelity_improvement = cirq.fidelity(a,c,atol=1e-6)-cirq.fidelity(a,b,atol=1e-6)

print(f"Fidelity Improvement % : {fidelity_improvement*100:.6f}")